# MODIS Surface Reflectance with OpenGeo (Planetary Computer)

This notebook demonstrates how to access **MODIS** data using the `opengeo` package with the **Microsoft Planetary Computer** STAC API, which provides a highly efficient and well-indexed version of the NASA data.

## 1. Initialize OpenGeo

In [ ]:
import opengeo as og

# Initialize with Microsoft Planetary Computer
og.Initialize("MICROSOFT")

## 2. Define ROI

Define a region over the Amazon Rainforest.

In [ ]:
roi = og.Geometry.Rectangle(-65, -5, -60, 0)
print("ROI:", roi)

## 3. Load MODIS Collection

Using `modis-09ga-061` on Planetary Computer. The bands on MS PC are usually named `sur_refl_b01`, etc.

In [ ]:
modis_col = og.ImageCollection("modis-09ga-061") \
    .filterDate("2023-01-01", "2023-01-10") \
    .filterBounds(roi) \
    .select(["sur_refl_b01", "sur_refl_b02"]) # Band 1: Red, Band 2: NIR

print(f"Images found: {modis_col.size()}")

## 4. Calculate NDVI

We use Dask for efficient computation.

In [ ]:
# Create median composite
modis_image = modis_col.median(epsg=3857)

ndvi = modis_image.normalizedDifference(["sur_refl_b02", "sur_refl_b01"]).rename("NDVI")

Map = og.Map()
Map.addLayer(ndvi, {'min': 0, 'max': 0.8, 'palette': 'red,yellow,green'}, "MODIS NDVI")
Map.centerObject(roi, zoom=5)
Map